In [2]:
!pip install geopy
from geopy.geocoders import Nominatim
import geopy
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
! pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
from bs4 import BeautifulSoup
import requests

In [7]:
Toronto_data = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(Toronto_data).text

In [8]:
soup = BeautifulSoup(source, 'xml')

In [9]:
table=soup.find('table')

In [10]:
# columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
neighborhood = pd.DataFrame(columns = column_names)

In [11]:
neighborhood

,Postalcode,Borough,Neighborhood


In [13]:
#  postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        neighborhood.loc[len(neighborhood)] = row_data

In [14]:
neighborhood.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


CLEANING DATA

In [15]:
drop_data = neighborhood[neighborhood.Borough != 'Not assigned'].reset_index(drop=True)

In [16]:
df_toronto = drop_data.groupby(['Postalcode','Borough'], as_index=False).agg(lambda x: ','.join(x))

In [18]:
mask = df_toronto['Neighborhood'] == "Not assigned"
df_toronto.loc[mask, 'Neighborhood'] = df_toronto.loc[mask, 'Borough']

In [19]:
df_toronto

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge,Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek,Rouge H..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill,Guildwood, M..."
3,M1G,Scarborough,"Woburn,Woburn"
4,M1H,Scarborough,"Cedarbrae,Cedarbrae"
5,M1J,Scarborough,"Scarborough Village,Scarborough Village"
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park,Ke..."
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge,Golden Mile, C..."
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village Wes..."
9,M1N,Scarborough,"Birch Cliff, Cliffside West,Birch Cliff, Cliff..."


In [20]:
df_toronto.shape

(103, 3)

SECOND SECTION

In [26]:
def get_geocode(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [28]:
df_geospatial=pd.read_csv('http://cocl.us/Geospatial_data')

In [30]:
df_geospatial.rename(columns={'Postal Code':'Postalcode'},inplace=True)
df_toronto = pd.merge(df_geospatial, df_toronto, on='Postalcode')

In [31]:
df_toronto.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge,Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek,Rouge H..."
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill,Guildwood, M..."
3,M1G,43.770992,-79.216917,Scarborough,"Woburn,Woburn"
4,M1H,43.773136,-79.239476,Scarborough,"Cedarbrae,Cedarbrae"


In [35]:
df_toronto=df_toronto[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [36]:
df_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge,Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek,Rouge H...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill,Guildwood, M...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn,Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae,Cedarbrae",43.773136,-79.239476


THIRD SECTION

In [37]:
CLIENT_ID = 'CONIN1VSHX2IAUQ31ZO0MFG5ONE044UXLI42ZUZKT2GS5MSX' # your Foursquare ID
CLIENT_SECRET = '2LJTYEOZELRXMWMRLONFPCTN10M2OKACID5IOJYOV2OE2YIH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CONIN1VSHX2IAUQ31ZO0MFG5ONE044UXLI42ZUZKT2GS5MSX
CLIENT_SECRET:2LJTYEOZELRXMWMRLONFPCTN10M2OKACID5IOJYOV2OE2YIH


In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Malvern, Rouge,Malvern, Rouge
Rouge Hill, Port Union, Highland Creek,Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill,Guildwood, Morningside, West Hill
Woburn,Woburn
Cedarbrae,Cedarbrae
Scarborough Village,Scarborough Village
Kennedy Park, Ionview, East Birchmount Park,Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge,Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West,Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West,Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre,Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale,Wexford, Maryvale
Agincourt,Agincourt
Clarks Corners, Tam O'Shanter, Sullivan,Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East,Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West,Steeles West, L'Amoreaux West
Upper Rouge,Upper Rouge
Hill

In [43]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge,Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek,Rouge H...",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek,Rouge H...",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill,Guildwood, M...",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill,Guildwood, M...",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Agincourt,Agincourt",5,5,5,5,5,5
"Alderwood, Long Branch,Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North,Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
"Bayview Village,Bayview Village",4,4,4,4,4,4
"Bedford Park, Lawrence Manor East,Bedford Park, Lawrence Manor East",22,22,22,22,22,22
"Berczy Park,Berczy Park",55,55,55,55,55,55
"Birch Cliff, Cliffside West,Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place,Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto,Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


In [47]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(2141, 273)

In [48]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Agincourt,Agincourt",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.

In [49]:
toronto_grouped.shape

(96, 273)

In [50]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt,Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1             Clothing Store   0.2
2             Breakfast Spot   0.2
3                     Lounge   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch,Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2  Sandwich Place  0.14
3             Pub  0.14
4     Coffee Shop  0.14


----Bathurst Manor, Wilson Heights, Downsview North,Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.10
1                       Bank  0.10
2                   Pharmacy  0.05
3              Shopping Mall  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village,Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence M

                             venue  freq
0                           Garden   0.5
1                      Music Venue   0.5
2                Accessories Store   0.0
3        Middle Eastern Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----Rouge Hill, Port Union, Highland Creek,Rouge Hill, Port Union, Highland Creek----
                        venue  freq
0  Construction & Landscaping   0.5
1                         Bar   0.5
2           Accessories Store   0.0
3   Middle Eastern Restaurant   0.0
4         Monument / Landmark   0.0


----Runnymede, Swansea,Runnymede, Swansea----
              venue  freq
0              Café  0.09
1       Coffee Shop  0.09
2  Sushi Restaurant  0.06
3               Pub  0.06
4       Pizza Place  0.06


----Runnymede, The Junction North,Runnymede, The Junction North----
               venue  freq
0           Bus Line  0.25
1  Convenience Store  0.25
2     Breakfast Spot  0.25
3            Brewery  0.25
4  Accessories Store  0.00


----Scarboro

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agincourt,Agincourt",Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Diner
1,"Alderwood, Long Branch,Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Sandwich Place,Coffee Shop,Pub,Dog Run,Dim Sum Restaurant,Diner,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview Nort...",Bank,Coffee Shop,Fried Chicken Joint,Chinese Restaurant,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket
3,"Bayview Village,Bayview Village",Café,Japanese Restaurant,Chinese Restaurant,Bank,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,"Bedford Park, Lawrence Manor East,Bedford Park...",Sandwich Place,Italian Restaurant,Coffee Shop,Greek Restaurant,Thai Restaurant,Liquor Store,Comfort Food Restaurant,Juice Bar,Butcher,Café


## Cluster Neighborhoods

In [80]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [84]:
toronto_merged.drop(['cluster labels'], axis=1)

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge,Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Women's Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek,Rouge H...",43.784535,-79.160497,1.0,Construction & Landscaping,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill,Guildwood, M...",43.763573,-79.188711,1.0,Breakfast Spot,Restaurant,Electronics Store,Rental Car Location,Medical Center,Intersection,Bank,Mexican Restaurant,Yoga Studio,Dog Run
3,M1G,Scarborough,"Woburn,Woburn",43.770992,-79.216917,1.0,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,M1H,Scarborough,"Cedarbrae,Cedarbrae",43.773136,-79.239476,1.0,Caribbean Restaurant,Hakka Restaurant,Bakery,Gas Station,Athletics & Sports,Bank,Thai Restaurant,Fried Chicken Joint,Doner Restaurant,Distribution Center
5,M1J,Scarborough,"Scarborough Village,Scarborough Village",43.744734,-79.239476,1.0,Playground,Smoke Shop,Jewelry Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park,Ke...",43.727929,-79.262029,1.0,Department Store,Hobby Shop,Coffee Shop,Train Station,Ethiopian Restaurant,Event Space,Escape Room,Falafel Restaurant,Electronics Store,Dim Sum Restaurant
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge,Golden Mile, C...",43.711112,-79.284577,1.0,Bus Line,Bakery,Park,Ice Cream Shop,Bus Station,Metro Station,Intersection,Soccer Field,Creperie,Dog Run
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village Wes...",43.716316,-79.239476,1.0,American Restaurant,Motel,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
9,M1N,Scarborough,"Birch Cliff, Cliffside West,Birch Cliff, Cliff...",43.692657,-79.264848,1.0,College Stadium,Skating Rink,General Entertainment,Café,Escape Room,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant


In [90]:
toronto_merged=toronto_merged.drop([16, 21,93],axis=0)

In [91]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, borough, neighbourhood in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Neighborhood']):
    label = '{}, {}'.format('Neighborhood', 'Borough')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters) 
       
map_clusters